## ■ Image Files resize (64x64)

In [3]:
import  cv2
import  os 
import  numpy  as np

path = "E:\\data11\\original"

file_list = os.listdir(path)
    
for k in file_list:
    img = cv2.imread(path + '\\' + k)
    width, height = img.shape[:2]
    resize_img = cv2.resize(img, (64 , 64), interpolation=cv2.INTER_CUBIC)
    cv2.imwrite('E:\\data11\\resize_64\\' + k, resize_img)

## ■ Setting

#### ▶ Train_data (1 ~ 7000) :  7000개

#### ▶ Test_data (7001 ~ 7470) : 470개

## ■ Create Labels

#### ● Change 2 dimension array to 1 dimension array

In [2]:
import csv

path1 = 'E:\\data11\\xray_labels.csv'
file = open(path1)

labeldata = csv.reader(file)
labellist = []

for  i   in  labeldata:
    labellist.append(i)

labellist2 = sum(labellist,[])

#### ● Change 'normal' = 1 , 'patient' = 0

In [3]:
path2 = 'E:\\data11\\xray_labels2.csv'
file2 = open(path2, 'w')

# normal = 1 , patient = 0

for  i  in  labellist2:
    if i == 'patient':
        file2.write( str(0) + '\n' )
    else:
        file2.write( str(1) + '\n' )
        
file2.close()

#### ● Save Train Labels and Test Labels

In [4]:
# save train labels

path = 'E:\\data11\\xray_labels2.csv'
file = open(path)

labeldata = csv.reader(file)
labellist = []

for  i   in  labeldata:
    labellist.append(i)

labellist2 = sum(labellist,[]) # sum 함수의 default 값은 0이므로 []로 바꾸어서 리스트 합침.

# print(labellist)


path2 = 'E:\\data11\\train_label.csv'
file2 = open(path2, 'w')

for  i  in  range(0,7000):
    file2.write( str(labellist2[i]) + '\n' )
    
file2.close()

In [5]:
# save test labels

path = 'E:\\data11\\xray_labels2.csv'
file = open(path)

labeldata = csv.reader(file)
labellist = []

for  i   in  labeldata:
    labellist.append(i)

labellist2 = sum(labellist,[]) # sum 함수의 default 값은 0이므로 []로 바꾸어서 리스트 합침.

# print(labellist)


path2 = 'E:\\data11\\test_label.csv'
file2 = open(path2, 'w')

for  i  in  range(7000,7470):
    file2.write( str(labellist2[i]) + '\n' )
    
file2.close()

## ■ Create loader4.py 

 ■ image_load  
 ■ next_batch  
 ■ shuffle_batch  
 ■ label_load  

In [6]:
import   csv
import  os
import  re
import  cv2
import  random
import  numpy  as  np

def  image_load(path):
    file_list = os.listdir(path)
    file_name = []
    for  i  in  file_list:
        a = int(  re.sub('[^0-9]', '', i )  ) # 숫자가 아닌것은 '' 로 처리 
        file_name.append(a) 
    file_name.sort()

    file_res = [] 
    for  j  in   file_name:
        file_res.append('%s\\%d.png' %(path,j)  )

    image = []
    for  k  in  file_res:
        img = cv2.imread(k)
        image.append(img)

    return  np.array(image)

def  label_load( path ):
    file = open(path)
    labeldata = csv.reader(file)
    labellist = []
    for  i   in  labeldata:
        labellist.append(i)

    label = np.array(labellist)
    label = label.astype(int)  # 숫자로 변환 
    label = np.eye(2)[label]
    label = label.reshape(-1,2) 
    return  label


def  shuffle_batch( data_list, label ):
    x = np.arange( len( data_list) )
    random.shuffle(x)
    data_list2 = data_list[x]
    label2 = label[x]
    return   data_list2, label2 


def  next_batch( data1, data2, init,  fina ):
    return  data1[ init : fina ],  data2[init : fina] 

#### ● Check train_data and test_data shape

In [4]:
import loader4

train_image='E:\\data11\\train_resize_64\\'
train_label= 'E:\\data11\\train_label.csv'
test_image='E:\\data11\\test_resize_64\\'
test_label='E:\\data11\\test_label.csv'

print(loader4.image_load(train_image).shape)
print(loader4.label_load(train_label).shape)
print(loader4.image_load(test_image).shape)
print(loader4.label_load(test_label).shape)

(7000, 64, 64, 3)
(7000, 2)
(470, 64, 64, 3)
(470, 2)


### ■ Train start !

In [1]:
from keras.models import Sequential, save_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np
from keras.utils import np_utils
from keras.layers.normalization import BatchNormalization
import loader4

batch_size = 28
num_classes = 2
epochs = 100
 
train_image='E:\\data11\\train_resize_64\\'
train_label= 'E:\\data11\\train_label.csv'
test_image='E:\\data11\\test_resize_64\\'
test_label='E:\\data11\\test_label.csv'

x_train = loader4.image_load(train_image)
y_train = loader4.label_load(train_label)
x_test = loader4.image_load(test_image)
y_test = loader4.label_load(test_label)      


print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(x_train.shape[1:])

#(x_train, y_train), (x_test, y_test) = cifar10.load_data()
# One hot Encoding



model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes))
model.add(Activation('softmax'))
 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
 
hist = model.fit(x_train, y_train, validation_data=(x_test, y_test), nb_epoch=epochs, batch_size=batch_size, verbose=1)
 
scores = model.evaluate(x_test, y_test, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))
 
save_model(model, "E:\data11\\models\\result_64.h5")

Using TensorFlow backend.


(7000, 64, 64, 3)
(7000, 2)
(470, 64, 64, 3)
(470, 2)
(64, 64, 3)


C:\anaconda3\envs\tf2.0-gpu\lib\site-packages\ipykernel_launcher.py:73: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 7000 samples, validate on 470 samples
Epoch 1/100
7000/7000 [==============================] - 8s 1ms/step - loss: 0.8204 - accuracy: 0.5886 - val_loss: 0.6874 - val_accuracy: 0.6043
Epoch 2/100
7000/7000 [==============================] - 5s 783us/step - loss: 0.6721 - accuracy: 0.6230 - val_loss: 0.6658 - val_accuracy: 0.6043
Epoch 3/100
7000/7000 [==============================] - 5s 783us/step - loss: 0.6492 - accuracy: 0.6376 - val_loss: 0.6508 - val_accuracy: 0.6064
Epoch 4/100
7000/7000 [==============================] - 5s 767us/step - loss: 0.6343 - accuracy: 0.6476 - val_loss: 0.6416 - val_accuracy: 0.6255
Epoch 5/100
7000/7000 [==============================] - 5s 771us/step - loss: 0.6249 - accuracy: 0.6497 - val_loss: 0.6570 - val_accuracy: 0.6277
Epoch 6/100
7000/7000 [==============================] - 5s 770us/step - loss: 0.6136 - accuracy: 0.6639 - val_loss: 0.6781 - val_accuracy: 0.6106
Epoch 7/100
7000/7000 [==============================] - 5s 766us/step - 

7000/7000 [==============================] - 5s 783us/step - loss: 0.0417 - accuracy: 0.9839 - val_loss: 1.1221 - val_accuracy: 0.6340
Epoch 57/100
7000/7000 [==============================] - 5s 782us/step - loss: 0.0458 - accuracy: 0.9841 - val_loss: 1.0079 - val_accuracy: 0.6426
Epoch 58/100
7000/7000 [==============================] - 5s 767us/step - loss: 0.0458 - accuracy: 0.9827 - val_loss: 1.1932 - val_accuracy: 0.6404
Epoch 59/100
7000/7000 [==============================] - 5s 759us/step - loss: 0.0479 - accuracy: 0.9829 - val_loss: 1.1597 - val_accuracy: 0.6213
Epoch 60/100
7000/7000 [==============================] - 5s 760us/step - loss: 0.0479 - accuracy: 0.9827 - val_loss: 1.1042 - val_accuracy: 0.6234
Epoch 61/100
7000/7000 [==============================] - 5s 761us/step - loss: 0.0426 - accuracy: 0.9864 - val_loss: 1.2659 - val_accuracy: 0.6511
Epoch 62/100
7000/7000 [==============================] - 5s 758us/step - loss: 0.0399 - accuracy: 0.9860 - val_loss: 1.1176 